## Handling Imbalanced Dataset

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import KFold
import numpy as np


C:\Users\Lenovo\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Lenovo\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
from sklearn.model_selection import GridSearchCV

In [3]:
df=pd.read_csv('creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df.shape

(284807, 31)

In [5]:
df.isnull().sum()           #null value check

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [6]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [7]:
#### Its a huge difference and highly imbalanced dataset

In [8]:
## Deviding into dependent and independent features
x=df.drop('Class',axis=1)
y=df.Class

### Test without sampling

In [9]:
# initialize logarthmic classification
log_class= LogisticRegression()

# perform hyperparameter tuning
grid={'C':10.0**np.arange(-2,3),'penalty':['l1','l2']}
cv=KFold(n_splits=5,random_state=None,shuffle=False)

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,train_size=0.7)

In [11]:
clf= GridSearchCV(log_class,grid,cv=cv,n_jobs=-1,scoring='f1_macro')
clf.fit(X_train,y_train)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro')

In [12]:
y_pred=clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85261    41]
 [   45    96]]
0.9989934810341398
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85302
           1       0.70      0.68      0.69       141

    accuracy                           1.00     85443
   macro avg       0.85      0.84      0.85     85443
weighted avg       1.00      1.00      1.00     85443



### Random forest using class weights (ensembeled technique)             

In [13]:
class_weight=dict({0:1,1:100})       #give valu 1 100 times more importance

In [15]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier(class_weight=class_weight)
classifier.fit(X_train,y_train)

C:\Users\Lenovo\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Lenovo\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


RandomForestClassifier(class_weight={0: 1, 1: 100})

In [16]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85295     7]
 [   36   105]]
0.9994967405170698
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85302
           1       0.94      0.74      0.83       141

    accuracy                           1.00     85443
   macro avg       0.97      0.87      0.91     85443
weighted avg       1.00      1.00      1.00     85443



### Under Sampling
By using under sampling i will try to reduce the maximum labels.
loss of data

In [17]:
y_train.value_counts()

0    199013
1       351
Name: Class, dtype: int64

In [21]:
from collections import Counter
from imblearn.under_sampling import NearMiss
ns=NearMiss(0.8)     # take 80 % of total parameters with respect to 1
X_train_ns,y_train_ns= ns.fit_sample(X_train,y_train)
print('The no. of class before the fit {}'.format(Counter(y_train)))
print('The no. of class after the fit {}'.format(Counter(y_train_ns)))

C:\Users\Lenovo\Anaconda3\lib\site-packages\imblearn\utils\_validation.py:638: FutureWarning: Pass sampling_strategy=0.8 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning)


The no. of class before the fit Counter({0: 199013, 1: 351})
The no. of class after the fit Counter({0: 438, 1: 351})


In [22]:
(80/100)*438                      #the value reduce to 438

350.40000000000003

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train_ns,y_train_ns)

In [ ]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

### Over Sampling

In [24]:
from imblearn.over_sampling import RandomOverSampler

In [30]:
os=RandomOverSampler(0.75)     
X_train_os,y_train_os= os.fit_sample(X_train,y_train)
print('The no. of class before the fit {}'.format(Counter(y_train)))
print('The no. of class after the fit {}'.format(Counter(y_train_os)))

C:\Users\Lenovo\Anaconda3\lib\site-packages\imblearn\utils\_validation.py:638: FutureWarning: Pass sampling_strategy=0.75 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning)


The no. of class before the fit Counter({0: 199013, 1: 351})
The no. of class after the fit Counter({0: 199013, 1: 149259})


In [31]:
## 99506 is 40% of 199013

In [33]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train_os,y_train_os)

KeyboardInterrupt: 

In [ ]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

### SMOTE

In [34]:
from imblearn.combine import SMOTETomek

In [35]:
os=SMOTETomek(0.5)     
X_train_os,y_train_os= os.fit_sample(X_train,y_train)
print('The no. of class before the fit {}'.format(Counter(y_train)))
print('The no. of class after the fit {}'.format(Counter(y_train_os)))

C:\Users\Lenovo\Anaconda3\lib\site-packages\imblearn\utils\_validation.py:638: FutureWarning: Pass sampling_strategy=0.5 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning)


The no. of class before the fit Counter({0: 199013, 1: 351})
The no. of class after the fit Counter({0: 198102, 1: 98595})


In [36]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train_os,y_train_os)

RandomForestClassifier()

In [38]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85281    21]
 [   29   112]]
0.9994148145547324
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85302
           1       0.84      0.79      0.82       141

    accuracy                           1.00     85443
   macro avg       0.92      0.90      0.91     85443
weighted avg       1.00      1.00      1.00     85443



### Ensembeled techniques

In [37]:
from imblearn.ensemble import EasyEnsembleClassifier

In [39]:
easy=EasyEnsembleClassifier()
easy.fit(X_train,y_train)

EasyEnsembleClassifier()

In [40]:
y_pred=easy.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[82513  2789]
 [   11   130]]
0.9672296150650141
              precision    recall  f1-score   support

           0       1.00      0.97      0.98     85302
           1       0.04      0.92      0.08       141

    accuracy                           0.97     85443
   macro avg       0.52      0.94      0.53     85443
weighted avg       1.00      0.97      0.98     85443

